In [1]:
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it")

class CountryModel(BaseModel):

    name : str = Field(description="Name of the Country")
    language: str = Field(description="Official Language of the country")
    capital: str = Field(description="Capital of the country")

structured_llm = llm.with_structured_output(CountryModel)
structured_llm

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10df8b890>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10dac9ad0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'CountryModel', 'description': '', 'parameters': {'properties': {'name': {'description': 'Name of the Country', 'type': 'string'}, 'language': {'description': 'Official Language of the country', 'type': 'string'}, 'capital': {'description': 'Capital of the country', 'type': 'string'}}, 'required': ['name', 'language', 'capital'], 'type': 'object'}}}], 'ls_structured_output_format': {'kwargs': {'method': 'function_calling'}, 'schema': {'type': 'function', 'function': {'name': 'CountryModel', 'description': '', 'parameters': {'properties': {'name': {'description': 'Name of the Country', 'type': 'string'}, 'language': {'description': 'Official Language of th

In [12]:
structured_llm.invoke("Tell me about Portugal")

CountryModel(name='Portugal', language='Portuguese', capital='Lisbon')

In [23]:
from typing_extensions import Annotated, TypedDict
from typing import Optional

class Joke(TypedDict):

    """Joke to tell user"""

    setup: Annotated[str, ..., "The SetUp of the Joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], 6, "How funny the joke is, from 1 to 10"]

structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a Dad joke")

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!',
 'rating': 6}

In [20]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about cats")

{'setup': "Why don't cats play poker in the jungle?",
 'punchline': 'Too many cheetahs!'}